In [ ]:
# default_exp g_utils

In [ ]:
#hide_input
import pivotpy as pp 
pp.nav_links(4)

<style>
                a{text-decoration: none;color:lightkblue;font-weight:bold;}
                a:focus,a:active.a:hover{color:hotpink;}
                </style>
**Navigation:** [Index&nbsp;&nbsp;](https://massgh.github.io/pivotpy/)
[XmlElementTree&nbsp;&nbsp;](https://massgh.github.io/pivotpy/XmlElementTree)
[StaticPlots&nbsp;&nbsp;](https://massgh.github.io/pivotpy/StaticPlots)
[InteractivePlots&nbsp;&nbsp;](https://massgh.github.io/pivotpy/InteractivePlots)
[●Utilities&nbsp;&nbsp;](https://massgh.github.io/pivotpy/Utilities)
[StructureIO&nbsp;&nbsp;](https://massgh.github.io/pivotpy/StructureIO)
[Widgets&nbsp;&nbsp;](https://massgh.github.io/pivotpy/Widgets)


# Utilities

> This includes generally useful functions, including running PowerShell commands from Python.

In [ ]:
#hide
from numpy.core.overrides import set_module

In [ ]:
#export
def get_file_size(path):
    import os
    if os.path.isfile(path):
        size = os.stat(path).st_size
        for unit in ['Bytes','KB','MB','GB','TB']:
            if size < 1024.0:
                return "%3.2f %s" % (size,unit)
            size /= 1024.0
    else:
        return ''

In [ ]:
get_file_size('Utilities.ipynb')

'24.41 KB'

In [ ]:
#export
def interpolate_data(x,y,n=10,k=3):
    """
    - Returns interpolated xnew,ynew. If two points are same, it will add 0.1*min(dx>0) to compensate it.
    - **Parameters**
        - x: 1D array of size p,
        - y: ndarray of size p*q*r,....
        - n: Number of points to add between two given points.
        - k: Polynomial order to interpolate.

    - Only axis 0 will be interpolated. If you want general interploation, use `from scipy.interpolate import make_interp_spline, BSpline`

    - **General Usage**: K(p),E(p,q) input from bandstructure.
        - `Knew,Enew= interpolate_data(K,E,n=10,k=3)`. cubic interploation
    """
    import numpy as np
    #Add very small values at simliar points to make interpolation work.
    ind=[i for i in range(0,len(x)) if x[i-1]==x[i]] #Duplicate indices
    xa=np.unique(x)
    dx=0.1*np.min(xa[1:]-xa[:-1])
    if(ind):
        for pt in ind:
            x[pt:]=x[pt:]-x[pt]+x[pt-1]+dx
    # Now Apply interpolation
    from scipy.interpolate import make_interp_spline, BSpline
    xnew=[np.linspace(x[i],x[i+1],n) for i in range(len(x)-1)]
    xnew=np.reshape(xnew,(-1))
    spl = make_interp_spline(x, y, k=k) #BSpline object
    ynew = spl(xnew)
    return xnew,ynew

In [ ]:
#export
def ps_to_py(ps_command='Get-ChildItem', exec_type='-Command', path_to_ps='powershell.exe'):
    """
    - Captures powershell output in python.
    - **Parameters**
        - ps_command: enclose ps_command in ' ' or " ".
        - exec_type : type of execution, default '-Command', could be '-File'.
        - path_to_ps: path to powerhell.exe if not added to PATH variables.
    """
    from subprocess import Popen, PIPE
    try: # Works on Linux and Windows if PS version > 5.
        cmd = ['pwsh', '-ExecutionPolicy', 'Bypass', exec_type, ps_command]
        proc = Popen(cmd, stdout=PIPE, stderr=PIPE)
    except FileNotFoundError:
        try: # Works only on Windows.
            cmd = ['powershell', '-ExecutionPolicy', 'Bypass', exec_type, ps_command]
            proc = Popen(cmd, stdout=PIPE, stderr=PIPE)
        except FileNotFoundError:
            # Works in case nothing above works and you know where is executable.
            cmd = [path_to_ps, '-ExecutionPolicy', 'Bypass', exec_type, ps_command]
            proc = Popen(cmd, stdout=PIPE, stderr=PIPE)

    out=[]; #save to out.
    while True:
        line = proc.stdout.readline()
        if line!=b'':
            line=line.strip()
            u_line=line.decode("utf-8")
            out.append(u_line)
        else:
            break
    out=[item for item in out if item!=''] #filter out empty lines
    return out

In [ ]:
#export
def ps_to_std(ps_command='Get-ChildItem', exec_type='-Command', path_to_ps='powershell.exe'):
    """
    - Prints powershell output in python std.
    - **Parameters**
        - ps_command: enclose ps_command in ' ' or " ".
        - exec_type: type of execution, default '-Command', could be '-File'.
        - path_to_ps: path to powerhell.exe if not added to PATH variables.
    """
    out=ps_to_py(path_to_ps=path_to_ps,exec_type=exec_type,ps_command=ps_command)
    for item in out:
        print(item)
    return None

- `ps_to_std` only outputs returns of powershell to python std.
- `ps_to_py`'s return could be manipulated in python.

In [ ]:
ps_to_std(ps_command='(Get-Process)[0..4]')

Handles  NPM(K)    PM(K)      WS(K)     CPU(s)     Id  SI ProcessName
-------  ------    -----      -----     ------     --  -- -----------
560      49    39564      75824      17.38  16184   1 AltC
271      17     6072      24920       1.67   6776   1 ApplicationFrameHost
147       9     1432       5860              5320   0 armsvc
416      22   300176      39328       7.36   3556   0 audiodg
331      17     4656       6532       2.19  14560   1 CastSrv


In [ ]:
pp=ps_to_py(ps_command='(Get-Process)[0..4]')
pp

['NPM(K)    PM(M)      WS(M)     CPU(s)      Id  SI ProcessName',
 '------    -----      -----     ------      --  -- -----------',
 '48    37.55      63.02       8.05   15160   1 AltC',
 '32    30.82      83.93       0.48   15984   1 ApplicationFrameHost',
 '8     1.56       5.47       0.00    5144   0 armsvc',
 '33    21.95      37.47       0.89    2424   1 BingWallpaperApp',
 '9     1.66       8.92       0.03   15632   1 browser_broker']

In [ ]:
#export
import os
import glob
#Selection of required project directories.
def select_dirs(path = os.getcwd(),include=[],exclude=[]):
    """
    - Returns selected directories recursively from a parent directory.
    - **Parameters**
        - path    : path to a parent directory, default is `"."`
        - include : list of keywords to include directories, avoid wildcards.
        - exclude : list of keywords to exclude directories, avoid wildcards.
    - **Returns**
        - Tuple of two elements, list of selcted directories and given path.
    """
    print('Use command `get_child_items()` instead for more flexibility.')
    list_dirs=[]; req_dirs=[];
    for filename in glob.iglob(path + '**/**', recursive=True):
        if os.path.isdir(filename):
            list_dirs.append(filename)
    for item in list_dirs:
        for check in include:
            if(check in item):
                if(path != os.getcwd()):
                    req_dirs.append(item.replace("\\","/"))
                if(path == os.getcwd()):
                    req_dirs.append('.'+(item.split(os.getcwd())[-1]).replace("\\","/"))
    for item in req_dirs.copy():
        for ex in exclude:
            if ex in item:
                req_dirs.remove(item)
    return (req_dirs,path.replace("\\","/"))
#Selction of files in selected directories.
def select_files(path=os.getcwd(),include=[],exclude=[]):
    """
    - Returns selected files from a given directory.
    - **Parameters**
        - path    : path to a parent directory, default is `"."`
        - include : list of keywords to include files, avoid wildcards.
        - exclude : list of keywords to exclude files, avoid wildcards.
    - **Returns**
        - Tuple of two elements, list of selcted files and given path.
    """
    print('Use command `get_child_items()` instead for more flexibility.')
    req_files=[]
    all_files=os.listdir(path)
    for file in all_files:
        for check in include:
                    if(check in file):
                        req_files.append(file)
    for item in req_files.copy():
        for ex in exclude:
            if ex in item:
                req_files.remove(item)
    return (req_files,path.replace("\\","/"))

In [ ]:
#export
def get_child_items(path = os.getcwd(),depth=None,recursive=True,include=[],exclude=[],filesOnly=False,dirsOnly= False):
    """
    - Returns selected directories/files recursively from a parent directory.
    - **Parameters**
        - path    : path to a parent directory, default is `"."`
        - depth   : int, subdirectories depth to get recursively, default is None to list all down.
        - recursive : If False, only list current directory items, if True,list all items recursively down the file system.
        - include : list or str of keywords to include directories/files, avoid wildcards.
        - exclude : list or str of keywords to exclude directories/files, avoid wildcards.
        - filesOnly : Boolean, if True, returns only files.
        - dirsOnly  : Boolean, if True, returns only directories.
    - **Returns**
        - GLOB : Tuple (children,parent), children is list of selected directories/files and parent is given path. Access by index of by `get_child_items().{children,path}`.
    """
    import os
    import glob
    import numpy as np
    from collections import namedtuple
    if include != None and type(include) == str:
        include = [include,]
    if exclude != None and type(exclude) == str:
        exclude = [exclude,]
    path = os.path.abspath(path) # important
    pattern = path + '**/**' # Default pattern
    if depth != None and type(depth) == int:
        pattern = path + '/'.join(['*' for i in range(depth+1)])
        if glob.glob(pattern) == []: #If given depth is more, fall back.
            pattern = path + '**/**' # Fallback to default pattern if more depth to cover all.
    glob_files = glob.iglob(pattern, recursive=recursive)
    if dirsOnly == True:
        glob_files = filter(lambda f: os.path.isdir(f),glob_files)
    if filesOnly == True:
        glob_files = filter(lambda f: os.path.isfile(f),glob_files)
    list_dirs=[]
    for g_f in glob_files:
        list_dirs.append(os.path.relpath(g_f,path))
    # Include check
    req_dirs=[]
    if include != []:
        for check in include:
            req_dirs.extend(list(filter(lambda f: check in f ,list_dirs)))
    elif include == []:
        req_dirs = list_dirs
    # Exclude check
    to_exclude = []
    if exclude != []:
        for ex in exclude:
            to_exclude.extend(list(filter(lambda f: ex in f ,req_dirs)))
        req_dirs = [r_d for r_d in req_dirs if r_d not in to_exclude]
    # Keep only unique
    req_dirs = list(np.unique(req_dirs))
    out_files = namedtuple('GLOB',['children','parent'])
    return out_files(req_dirs,os.path.abspath(path))

In [ ]:
import pivotpy.g_utils as gu
dirs,fromdir=gu.select_dirs( path= "E:/Research/III_V_Interface/All_Interfaces" ,include=['mal'],exclude=['oped','test'])
print(fromdir)
dirs

Use command `get_child_items() instead for more flexibility.
E:/Research/III_V_Interface/All_Interfaces


['E:/Research/III_V_Interface/All_Interfaces/AlAs_Bi/primitive/Normal',
 'E:/Research/III_V_Interface/All_Interfaces/AlSb_Bi/primitive/Normal',
 'E:/Research/III_V_Interface/All_Interfaces/GaAs_Bi/primitive/Normal',
 'E:/Research/III_V_Interface/All_Interfaces/GaSb_Bi/primitive/Normal',
 'E:/Research/III_V_Interface/All_Interfaces/InAs_Bi/primitive/Normal',
 'E:/Research/III_V_Interface/All_Interfaces/InSb_Bi/primitive/Normal']

In [ ]:
files,fromdir=gu.select_files(path=".",include=['ipynb'],exclude=['check','Plots'])
print(fromdir)
files

Use command `get_child_items() instead for more flexibility.
.


['index.ipynb',
 'Structure IO.ipynb',
 'Utilities.ipynb',
 'XmlElementTree.ipynb']

In [ ]:
items = get_child_items(path="../",include='Current',exclude=['Jupyter'],dirsOnly=True)
print(items.parent)
items.children

e:\Research


['Current', 'Current\\DOS', 'Current\\__pycache__']

In [ ]:
#export
def invert_color(color=(1,1,1)):
    """
    - Returns opposite of given complementary color.
    - Input: Tuple (r,g,b).
    """
    r = min(color)+max(color)
    return tuple(r-c for c in color)

In [ ]:
invert_color((0.5,1,1))

(1.0, 0.5, 0.5)

In [ ]:
#export
def printr(s): print("\033[91m {}\033[00m" .format(s))
def printg(s): print("\033[92m {}\033[00m" .format(s))
def printb(s): print("\033[34m {}\033[00m" .format(s))
def printy(s): print("\033[93m {}\033[00m" .format(s))
def printm(s): print("\033[95m {}\033[00m" .format(s))
def printc(s): print("\033[96m {}\033[00m" .format(s))

In [ ]:
printm('######|||||||||||+++++|||||||||||||######') 
printr('######|||||||||||+++++|||||||||||||######') 
printy('######|||||||||||+++++|||||||||||||######') 
printg('######|||||||||||+++++|||||||||||||######') 
printc('######|||||||||||+++++|||||||||||||######') 
printb('######|||||||||||+++++|||||||||||||######') 
printm('######|||||||||||+++++|||||||||||||######')

 ######|||||||||||+++++|||||||||||||######
 ######|||||||||||+++++|||||||||||||######
 ######|||||||||||+++++|||||||||||||######
 ######|||||||||||+++++|||||||||||||######
 ######|||||||||||+++++|||||||||||||######
 ######|||||||||||+++++|||||||||||||######
 ######|||||||||||+++++|||||||||||||######


In [ ]:
#export
import json
class EncodeFromNumpy(json.JSONEncoder):
    """
    - Serializes python/Numpy objects via customizing json encoder.
    - **Usage**
        - `json.dumps(python_dict, cls=EncodeFromNumpy)` to get json string.
        - `json.dump(*args, cls=EncodeFromNumpy)` to create a file.json.
    """
    def default(self, obj):
        import numpy
        if isinstance(obj, numpy.ndarray):
            return {
                "_kind_": "ndarray",
                "_value_": obj.tolist()
            }
        if isinstance(obj, numpy.integer):
            return int(obj)
        elif isinstance(obj, numpy.floating):
            return float(obj)
        elif isinstance(obj,range):
            value = list(obj)
            return {
                "_kind_" : "range",
                "_value_" : [value[0],value[-1]+1]
            }
        return super(EncodeFromNumpy, self).default(obj)



class DecodeToNumpy(json.JSONDecoder):
    """
    - Deserilizes JSON object to Python/Numpy's objects.
    - **Usage**
        - `json.loads(json_string,cls=DecodeToNumpy)` from string, use `json.load()` for file.
    """
    def __init__(self, *args, **kwargs):
        json.JSONDecoder.__init__(self, object_hook=self.object_hook, *args, **kwargs)

    def object_hook(self, obj):
        import numpy
        if '_kind_' not in obj:
            return obj
        kind = obj['_kind_']
        if kind == 'ndarray':
            return numpy.array(obj['_value_'])
        elif kind == 'range':
            value = obj['_value_']
            return range(value[0],value[-1])
        return obj

In [ ]:
import json
import pivotpy as pp
vr=pp.export_vasprun(path = "E:/Research/graphene_example/ISPIN_2/bands/vasprun.xml" ,skipk=88,elim=[0,5])
vr.pop('xml',None)
print("ORIGINAL DATA : {}\n".format(vr.poscar))
s=json.dumps(vr.poscar, cls=EncodeFromNumpy)
print("JSON STRING : {}\n".format(s))
s2 = json.loads(s, cls=DecodeToNumpy)
print("RETRIEVED DATA : {}".format(s2))

ORIGINAL DATA : {'volume': 105.49324928, 'basis': array([[ 2.468031,  0.      ,  0.      ],
       [-1.234015,  2.137377,  0.      ],
       [ 0.      ,  0.      , 19.998293]]), 'rec_basis': array([[0.4051813 , 0.2339315 , 0.        ],
       [0.        , 0.46786318, 0.        ],
       [0.        , 0.        , 0.05000427]]), 'positions': array([[0.333333, 0.666667, 0.      ],
       [0.666667, 0.333333, 0.      ]])}

JSON STRING : {"volume": 105.49324928, "basis": {"_kind_": "ndarray", "_value_": [[2.468031, 0.0, 0.0], [-1.234015, 2.137377, 0.0], [0.0, 0.0, 19.998293]]}, "rec_basis": {"_kind_": "ndarray", "_value_": [[0.4051813, 0.2339315, 0.0], [0.0, 0.46786318, 0.0], [0.0, 0.0, 0.05000427]]}, "positions": {"_kind_": "ndarray", "_value_": [[0.333333, 0.666667, 0.0], [0.666667, 0.333333, 0.0]]}}

RETRIEVED DATA : {'volume': 105.49324928, 'basis': array([[ 2.468031,  0.      ,  0.      ],
       [-1.234015,  2.137377,  0.      ],
       [ 0.      ,  0.      , 19.998293]]), 'rec_basis':

## Add a function as method to a class without defining inside class itself
- We can add a function `f` to a class `C` if function includes parameter `self` as shown below:
```python
def f(self):
     print('Do something!')
setattr(C, 'f', f)
```
- We can define a wrapper to attach a method to class without keeping `self` as argument too.

In [ ]:
#export
def link_to_class(cls):
    """
    - Binds wrapper of a function to class as attribute that does exactly the same as function. Also function returned from wrapper can be used normally as well.
    - **Parameters**
        - cls : A class object to which function is attached.
    """
    from functools import wraps
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            return func(*args, **kwargs)
        setattr(cls, func.__name__, wrapper)
        return func
    return decorator

In [ ]:
# See `link_to_class` in action.
class Parent:
    pass
a = Parent()
try:
    a.f()
except AttributeError as ae:
    print(ae) 

# Now lets add f to Parent.
@link_to_class(Parent)
def f(s):
    print(s)

a.f('I am from class Parent!')
f('I am independent function!')
print(a.f) 
print(f) 

'Parent' object has no attribute 'f'
I am from class Parent!
I am independent function!
<bound method f of <__main__.Parent object at 0x00000225228DCFC8>>
<function f at 0x00000225230609D8>


In [ ]:
#export
def nav_links(current_index=0,
            doc_url = r"https://massgh.github.io/pivotpy/",
            items   = ["Index",
                       "XmlElementTree",
                       "StaticPlots",
                       "InteractivePlots",
                       "Utilities",
                       "StructureIO",
                       "Widgets"
                       ]):
    from IPython.display import Markdown,HTML
    links   = [doc_url+item if not 'Index' in item else doc_url for item in items] 
    style = """<style>
                a{text-decoration: none;color:lightkblue;font-weight:bold;}
                a:focus,a:active.a:hover{color:hotpink;}
                </style>\n"""
    md_str = style + ">>>> **Navigation:** "
    for i,(link,item) in enumerate(zip(links,items)):
        if current_index == i: item = "{}●".format(item)
        md_str += "[`▶`{}&nbsp;&nbsp;]({})\n".format(item,link)
    return Markdown(md_str+"\n-----")

In [ ]:
#hide_input
from IPython.display import Markdown
nav_links(4)

<style>
                a{text-decoration: none;color:lightkblue;font-weight:bold;}
                a:focus,a:active.a:hover{color:hotpink;}
                </style>
>>>> **Navigation:** [`▶`Index&nbsp;&nbsp;](https://massgh.github.io/pivotpy/)
[`▶`XmlElementTree&nbsp;&nbsp;](https://massgh.github.io/pivotpy/XmlElementTree)
[`▶`StaticPlots&nbsp;&nbsp;](https://massgh.github.io/pivotpy/StaticPlots)
[`▶`InteractivePlots&nbsp;&nbsp;](https://massgh.github.io/pivotpy/InteractivePlots)
[`▶`Utilities●&nbsp;&nbsp;](https://massgh.github.io/pivotpy/Utilities)
[`▶`StructureIO&nbsp;&nbsp;](https://massgh.github.io/pivotpy/StructureIO)
[`▶`Widgets&nbsp;&nbsp;](https://massgh.github.io/pivotpy/Widgets)

-----